* OpenAI Whisper 可以把影片加字幕，但研究、訪談採取逐字稿的情景則不太合用，因無法辨識說話者，不會加標點。
* 這個程式用 Whisper 把語音轉正體中文逐字稿、每句結束加上全形逗號（但只會加逗號），再儲存成「原檔名_output.txt」
* 你可把文字檔下載後，截去不需要的部分、分段、加上合適標點（句號、驚歎號等）。

# 安裝及檢查必要的程式庫

執行每一格都按下「shift + Enter」它會執行這一格並跳到下一格，完成執行後左方會顯示數字而不是箭頭，這些是使用到的程式庫，如果沒安裝，它檢查完會去安裝，如果已經安裝了很快就跳到下一格。

* Whisper 聽打需要安裝 Whisper 以及 ffmpeg。
* Whisper 基本會轉換簡中，用 OpenCC 轉成正題中文。
* 測試過用 C++ 寫的 opencc，某些依賴項放在 OS 中有可能找不到，用 opencc_python_reimplemented 解決此問題。

In [ ]:
!pip install -U --quiet openai-whisper

In [ ]:
!pip install --quiet ffmpeg-python

In [ ]:
!pip install --quiet opencc_python_reimplemented

# 找到要轉檔的檔案路徑

存入你要轉檔的原檔案，通常是 MP3 格式。

* 如果你在本機使用 Anaconda/ JupyterLab，就把 mp3 檔放在檔案夾；
* 如果在 Colab 就要在側邊欄上傳 mp3。 

確定檔案擺好後執行這一格，找到你的 mp3 檔路徑（例如：./filename.mp3），複製下來。

In [ ]:
# 獲得相對路徑
import os

# Root dir
root_directory = "."

# travel all the dirs/ files
for root, directories, files in os.walk(root_directory):
    # root：current dir
    # directories：sub dirs
    # files：files list

    # 输出当前目录路径
    print("Current Dir:", root)

    # 输出当前目录中的文件
    for file in files:
        file_path = os.path.join(root, file)
        print("File:", file_path)

# 導入必要的庫

In [ ]:
import whisper
import time
from IPython.display import display, clear_output
import re
import opencc

# 進行聽打

In [ ]:
# 定義開始時間
start_time = time.time()

# 在 Mac/ Linux 的檔案路徑（從側邊複製相對路徑即可）
input_file_path = r"./file_to_proceed.mp3"
# 在 Windows 的檔案路徑（相對路徑常出錯，從檔案總管獲取絕對路徑，要從硬碟開始的全部）
# input_file_path = r"c:Mydocument/20240606陳勝騰meeting.mp3"

# 載入 Whisper 模型
model = whisper.load_model("medium")  # 有 small / medium / large，依照你的電腦算力選擇，large 常常會非常慢

# 進行聽打，指定語言為中文（不指定會翻成英文）
try:
    result = model.transcribe(input_file_path, language="zh")  # 指定語言為中文
except Exception as e:
    clear_output(wait=True)
    display(f"識別過程中出現錯誤: {str(e)}")
    raise

# 計算總執行時間
end_time = time.time()
elapsed_time = end_time - start_time
display(f"識別過程共花費了 {elapsed_time:.2f} 秒")

# 獲取聽打後的本文和分段資料
recognized_text = result["text"]
segments = result.get("segments", [])

# 用 opencc 將簡中轉換為正中（default 簡中）
converter = opencc.OpenCC('s2t')
recognized_text = converter.convert(recognized_text)

# 在每個段落結尾添加中文全形逗號（只能逗號）
new_recognized_text = ""
for segment in segments:
    # 獲取段落文本
    segment_text = segment["text"]
    # 將段落文本轉換為繁體中文
    segment_text = converter.convert(segment_text)
    # 在段落尾部添加全形逗號
    new_recognized_text += segment_text + "，"

# 印出添加全形逗號後的文本，檢查是否出錯
print("添加全形逗號後的文本：")
print(new_recognized_text)

# 生成輸出文件路徑
input_file_name = os.path.basename(input_file_path)
input_file_name_without_ext = os.path.splitext(input_file_name)[0]
output_file_name = f"{input_file_name_without_ext}_output.txt"
output_file_path = os.path.join(os.path.dirname(input_file_path), output_file_name)

# 將文本保存到「原檔名_output.txt」txt 文件
with open(output_file_path, "w", encoding="utf-8") as file:
    file.write(new_recognized_text)

print("文本已保存到", output_file_path)

# 下載完成的文字檔
最後，從左側邊欄的檔案夾符號裡下載「原檔名_output.txt」即可使用。